In [1]:
import os
os.chdir("/RAFT-Stereo/RGBNIR")

## SceneFlow dataset Sample downlwoad
Below is sample code for downloading the SceneFlow dataset. <br>
Note that the sample dataset does not include labels for the right image, <br>
so the augmentation process is not fully supported. <br>
Therefore, this code is intended for testing purposes only. <br>
To create training code, download the full SceneFlow dataset from 
[here](https://lmb.informatik.uni-freiburg.de/resources/datasets/SceneFlowDatasets.en.html). <br>
Please note that the full dataset may exceed 1 TB in size.  

In [ ]:
import os
import tarfile
import urllib.request
from tqdm import tqdm 

# Define the URL and target folder
url = "https://lmb.informatik.uni-freiburg.de/resources/datasets/SceneFlow/assets/Sampler.tar.gz"
target_folder = "."

# Create the target folder if it doesn't exist
os.makedirs(target_folder, exist_ok=True)

# Define the path for the downloaded tar.gz file
tar_file_path = os.path.join(target_folder, "Sampler.tar.gz")

# Progress bar hook
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

# Download the file with progress bar
print("Downloading file...")
with DownloadProgressBar(unit="B", unit_scale=True, miniters=1, desc="Sampler.tar.gz") as t:
    urllib.request.urlretrieve(url, tar_file_path, reporthook=t.update_to)
print("Download complete.")

# Extract the tar.gz file
print("Extracting files...")
with tarfile.open(tar_file_path, "r:gz") as tar:
    tar.extractall(path=target_folder)
print("Extraction complete.")

# Remove the tar.gz file after extraction
os.remove(tar_file_path)
print("Cleanup complete. Files are ready in:", target_folder)


## DataAugmentor Class

This class provides a framework for augmenting a dataset containing RGB and NIR stereo image pairs. The augmentation pipeline includes:

1. **Path Walker**: Detects all frame directories in the dataset.
2. **RGB-NIR Augment Renderer**: Implements augmentation techniques proposed in our supplemental document ([arXiv:2411.18025](https://arxiv.org/abs/2411.18025)).
3. **Batch Augmentation with Progress Bar**: Applies the augmentations across the dataset with real-time progress visualization.


In [2]:
from dataloader.augmentor import SceneFlowAugmentRender
sceneflow_augmentor = SceneFlowAugmentRender()

/RAFT-Stereo/RGBNIR/pfmread.py:5: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.0)
  from scipy import misc


## Sample rendering code


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

device = "cuda"
# K=np.array([[450, 0.0, 479.5], [0.0, 450, 269.5], [0.0, 0.0, 1.0]]) # For Driving3d dataset with 15mm focal length
K = np.array(
    [[1050.0, 0.0, 479.5], [0.0, 1050.0, 269.5], [0.0, 0.0, 1.0]]
)  # For Other datasets

SCENEFLOW_ROOT = "Sampler/FlyingThings3D/normal_map"

N = 3
plt.figure(figsize=(16, N * 3))
for i in range(0, N):
    idx = i + 6
    K_tensor = torch.from_numpy(K).to(device)
    item_left, item_right = sceneflow_augmentor.read_sceneflow_frame(
        f"{SCENEFLOW_ROOT}/left/{str(idx).zfill(4)}.pfm", "normal_map", K,
        right_label=False
    )

    shaded_left, shaded_right = sceneflow_augmentor.random_light_render(
        K, 1, item_left, item_right
    )
    shaded_nir, shaded_nir_right = sceneflow_augmentor.random_nir_render(
        item_left, item_right, K_tensor, intensity_range=[100, 1000]
    )
    plt.subplot(N, 4, i * 4 + 1)
    plt.imshow(item_left.image.cpu().numpy().astype(np.uint8))
    plt.axis("off")
    plt.subplot(N, 4, i * 4 + 2)
    plt.imshow(shaded_left.cpu().numpy())
    plt.axis("off")
    plt.subplot(N, 4, i * 4 + 3)
    plt.imshow(shaded_nir.cpu().numpy(), cmap="gray", vmin=0, vmax=1)
    plt.axis("off")
    plt.subplot(N, 4, i * 4 + 4)
    plt.imshow(K[0, 0] / item_left.depth_map.cpu(), cmap="magma", vmin=0, vmax=48)
    plt.axis("off")
plt.tight_layout()
plt.show()

## Full dataset augmentation loop code

In [5]:
from dataloader.augmentor import SceneFlowConfig
ROOT = "_YOUR_STORAGE_"
ROOT = "/bean/flyingthings3d/frames_cleanpass/TRAIN/"
paths = sceneflow_augmentor.sceneflow_search_frames(ROOT)
print(len(paths))
config = SceneFlowConfig()
config.term = "frames_cleanpass"
config.nir_intensity_range = [100,500]
config.K_tensor = torch.from_numpy(
    np.array([[450, 0.0, 479.5], [0.0, 450, 269.5], [0.0, 0.0, 1.0]])
)  # For Driving3d dataset with 15mm focal length
config.K_tensor = torch.from_numpy(
    np.array([[1050.0, 0.0, 479.5], [0.0, 1050.0, 269.5], [0.0, 0.0, 1.0]])
)  # For Driving3d dataset with 15mm focal length

sceneflow_augmentor.sceneflow_dataset_augment(paths, config)

22390


  0%|          | 0/22390 [00:00<?, ?it/s]

../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1232,0,0], thread: [32,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1232,0,0], thread: [33,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1232,0,0], thread: [34,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1232,0,0], thread: [35,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1232,0,0], thread: [36,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1232,0,0], thread: [37,0,0] Asse

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
